# EDA on orangutans behavior dataset

> Explicar un poco el dataset y el estudio
## Data preparation
> Explicar cómo está estructurado el dataset, y cómo pensamos modificarlo

    - Cambiar Sujeto a números
    - Cambiar Receptor a números
    - Cambiar Periodo a números
    - Rellenar Frecuencia NaN con 0
    - Drop las columnas Ubicación y Observaciones


In [1]:
import pandas as pd
import numpy as np

xls = pd.ExcelFile(r'./Registro orangutanes Barcelona.xlsx')
df_raw = pd.read_excel(xls,'Grupo')
df_raw.head()

,Fecha,Sujeto,Periodo,Registro diario,Conducta,Receptor,Frecuencia,Duración,Ubicacion,Observaciones
0,2019-04-15,Karl,Prejuego,1,Cam,NaN,NaN,00:00:15,vg2,NaN
1,2019-04-15,Karl,Prejuego,1,Forr,NaN,NaN,00:00:34,vg2,NaN
2,2019-04-15,Karl,Prejuego,1,Sen,NaN,NaN,00:00:52,vg4,NaN
3,2019-04-15,Karl,Prejuego,1,Cam,NaN,NaN,00:00:05,vg4,NaN
4,2019-04-15,Karl,Prejuego,1,Sen,NaN,NaN,00:00:34,vg4,NaN


- Drop las columnas Ubicación y Observaciones

In [2]:
df_raw=df_raw.drop(labels=["Observaciones","Ubicacion"], axis=1)

- Quitar asteriscos y cosas raras de 'Receptor'

In [3]:
df_raw.loc[df_raw['Receptor'] == 'Gibon*', 'Receptor'] = 'Gibon'
df_raw.loc[df_raw['Receptor'] == 'Storma*', 'Receptor'] = 'Storma'
df_raw.loc[df_raw['Receptor'] == 'Locky*', 'Receptor'] = 'Locky'
df_raw.loc[df_raw['Receptor'] == 'Hadia*', 'Receptor'] = 'Hadia'
df_raw.loc[df_raw['Receptor'] == 'Popo*', 'Receptor'] = 'Popo'
df_raw.loc[df_raw['Receptor'] == 'Jawie*', 'Receptor'] = 'Jawie'
df_raw.loc[df_raw['Receptor'] == 'jawie', 'Receptor'] = 'Jawie'


df_raw['Receptor'].value_counts()

Hadia                 552
Storma                527
Jawie                 520
Locky                 446
Popo                  391
                       68
Gibon                  40
Locky Storma           25
Karl                   21
Popo Hadia             20
Storma Hadia           19
Jawie Hadia            17
Storma Popo            15
Locky Popo             15
Jawie Storma           11
Jawie Popo              9
Storma Jawie            7
Locky Hadia             5
Locky Jawie             3
Storma Jawie Hadia      3
Locky Popo Hadia        2
Storma Popo Hadia       2
Locky Storma Jawie      2
Popo Gibon              1
Todos                   1
Popo Hadia*             1
Storma Jawie*           1
Storma  Jawie Popo      1
Locky Storma Hadia      1
locky Storma            1
Locky Jawie Storma      1
Name: Receptor, dtype: int64

- Pasar de cadenas a listas de nombres

In [4]:
df_raw['Receptor'] = df_raw['Receptor'].fillna('')
df_raw['Receptor'] = df_raw.Receptor.apply(lambda x : x.split())
df_raw['Receptor'].value_counts()

[]                        5454
[Hadia]                    552
[Storma]                   527
[Jawie]                    520
[Locky]                    446
[Popo]                     391
[Gibon]                     40
[Locky, Storma]             25
[Karl]                      21
[Popo, Hadia]               20
[Storma, Hadia]             19
[Jawie, Hadia]              17
[Locky, Popo]               15
[Storma, Popo]              15
[Jawie, Storma]             11
[Jawie, Popo]                9
[Storma, Jawie]              7
[Locky, Hadia]               5
[Locky, Jawie]               3
[Storma, Jawie, Hadia]       3
[Locky, Popo, Hadia]         2
[Storma, Popo, Hadia]        2
[Locky, Storma, Jawie]       2
[Popo, Gibon]                1
[Todos]                      1
[Popo, Hadia*]               1
[Storma, Jawie*]             1
[Storma, Jawie, Popo]        1
[Locky, Storma, Hadia]       1
[locky, Storma]              1
[Locky, Jawie, Storma]       1
Name: Receptor, dtype: int64

- convertir los sujetos de string a int

In [5]:
df_raw.loc[df_raw['Sujeto'] == 'Karl', 'Sujeto'] = 0
df_raw.loc[df_raw['Sujeto'] == 'Locky', 'Sujeto'] = 1
df_raw.loc[df_raw['Sujeto'] == 'Jawie', 'Sujeto'] = 2
df_raw.loc[df_raw['Sujeto'] == 'Storma', 'Sujeto'] = 3
df_raw.loc[df_raw['Sujeto'] == 'Popo', 'Sujeto'] = 4
df_raw.loc[df_raw['Sujeto'] == 'Hadia', 'Sujeto'] = 5

- Convertir los valores Nan de la columna frecuencia a 0

In [6]:
df_raw['Frecuencia'] = df_raw['Frecuencia'].fillna(0)

- Limpiar duración y pasarlo todo a datetime
  

In [9]:
df_raw.drop(df_raw[df_raw['Duración'] == 'd'].index, inplace=True)

In [10]:
import datetime as dt
df_raw['Duración'] = df_raw['Duración'].fillna(dt.time(0, 0))

In [11]:
df_raw[ "Duración"] = np.where(df_raw["Duración"].astype(str).str.fullmatch(r'\s*'),
                               dt.time(0,0),
                               df_raw['Duración'])

In [12]:
def to_seconds(t):
    try:
        return pd.to_timedelta(str(t))
    except:
        return np.nan

df_raw['Duración'] = df_raw['Duración'].apply(to_seconds)

In [13]:
df_raw['Duración'].sum()

Timedelta('3 days 17:05:14.088300')

- Agrupar por duración

In [15]:
df_raw['Conducta'] = df_raw['Conducta'].str.lower()
df_raw.groupby(['Fecha', 'Registro diario', 'Conducta', 'Sujeto'])['Duración'] \
.sum()\
.unstack('Conducta')\
.fillna(pd.Timedelta(seconds=0))

Conducta                                     alim         alogroo  \
Fecha      Registro diario Sujeto                                   
2019-04-15 1               0      0 days 00:00:34 0 days 00:00:00   
                           1      0 days 00:04:39 0 days 00:00:00   
                           2      0 days 00:02:31 0 days 00:00:00   
                           3      0 days 00:00:00 0 days 00:06:14   
                           4      0 days 00:00:00 0 days 00:00:00   
...                                           ...             ...   
2019-07-19 2               1      0 days 00:07:06 0 days 00:00:00   
                           2      0 days 00:00:00 0 days 00:00:00   
                           3      0 days 00:00:54 0 days 00:00:00   
                           4      0 days 00:00:21 0 days 00:00:00   
                           5      0 days 00:07:23 0 days 00:00:00   

Conducta                                    aprox             bra  \
Fecha      Registro diario Sujeto                                   
2019-04-15 1               0      0 days 00:00:00 0 days 00:00:00   
                           1      0 days 00:00:00 0 days 00:00:00   
                           2      0 days 00:00:00 0 days 00:00:00   
                           3      0 days 00:00:00 0 days 00:00:00   
                           4      0 days 00:00:00 0 days 00:00:00   
...                                           ...             ...   
2019-07-19 2               1      0 days 00:00:00 0 days 00:00:00   
                           2      0 days 00:00:00 0 days 00:00:19   
                           3      0 days 00:00:06 0 days 00:00:00   
                           4      0 days 00:00:00 0 days 00:00:00   
                           5      0 days 00:00:00 0 days 00:00:03   

Conducta                                      cam   chup    con   corr   cuad  \
Fecha      Registro diario Sujeto                                               
2019-04-15 1               0      0 days 00:02:21 0 days 0 days 0 days 0 days   
                           1      0 days 00:01:17 0 days 0 days 0 days 0 days   
                           2      0 days 00:01:08 0 days 0 days 0 days 0 days   
                           3      0 days 00:00:00 0 days 0 days 0 days 0 days   
                           4      0 days 00:00:00 0 days 0 days 0 days 0 days   
...                                           ...    ...    ...    ...    ...   
2019-07-19 2               1      0 days 00:00:30 0 days 0 days 0 days 0 days   
                           2      0 days 00:00:00 0 days 0 days 0 days 0 days   
                           3      0 days 00:00:46 0 days 0 days 0 days 0 days   
                           4      0 days 00:00:00 0 days 0 days 0 days 0 days   
                           5      0 days 00:00:10 0 days 0 days 0 days 0 days   

Conducta                                     forr  ...             pub  \
Fecha      Registro diario Sujeto                  ...                   
2019-04-15 1               0      0 days 00:01:32  ... 0 days 00:00:00   
                           1      0 days 00:00:15  ... 0 days 00:00:00   
                           2      0 days 00:01:40  ... 0 days 00:00:00   
                           3      0 days 00:00:00  ... 0 days 00:00:00   
                           4      0 days 00:00:07  ... 0 days 00:00:00   
...                                           ...  ...             ...   
2019-07-19 2               1      0 days 00:00:31  ... 0 days 00:00:02   
                           2      0 days 00:00:00  ... 0 days 00:00:00   
                           3      0 days 00:01:32  ... 0 days 00:00:00   
                           4      0 days 00:06:56  ... 0 days 00:00:00   
                           5      0 days 00:00:10  ... 0 days 00:00:00   

Conducta                                      ras            roba  \
Fecha      Registro diario Sujeto                                   
2019-04-15 1               0      0 days 00:00:04

**DataFrame limpio**
- Pasar nombres de columnas a inglés. 
- Tal como está arriba, pero con los nombres
- Pasar frecuencia a duración
- Eliminar frecuencia
- Limpiar duración y pasarlo a segundos
  
**DataFrame para visualización**
- Base: DF limpio
- Sujeto string
- Periodo string
- Fecha
- Agrupar por macroconductas
  - Positivas:
    - Conductas lúdicas
    - Lúdico-sociales
    - Afiliativas
  - Neutras:
    - Reposo
    - Locomoción
    - Alimentación
    - Higiene
    - Conductas de exploración
  - Negativas:
    - Agonísticas
    - Interacción con el público
  - Sexuales

**DataFrame para ML**
- Sujeto one hot
- Periodo one hot
- conductas sumadas por registro
  
**DataFrame para visualización de grafo social**
- Generar matriz adyacente con todas las relaciones entre ellos
- Sujeto 
- Receptor
